In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time

In [ ]:
BASE_URL = "https://www.olx.com.pk/"

In [ ]:
def land_page(url):
    driver.get(url)

In [ ]:
def change_location(location):
    location_bar = driver.find_element(By.CSS_SELECTOR, 'input[autocomplete="location-search"]')
    location_bar.click()
    location_bar.send_keys(Keys.CONTROL + 'a')
    location_bar.send_keys(Keys.BACK_SPACE)
    location_bar.send_keys(location)
    time.sleep(1)
    dropDown = driver.find_element(By.CSS_SELECTOR, 'div[aria-label="Locations dropdown"]')
    options = dropDown.find_elements(By.TAG_NAME, 'div')
    options[1].click()

In [ ]:
def search_item(item):
    search_bar = driver.find_element(By.CSS_SELECTOR, 'input[autocomplete="free-text-search"]')
    search_bar.clear()
    search_bar.send_keys(item)
    search_btn = driver.find_element(By.CSS_SELECTOR, 'button[aria-label="Search"]')
    search_btn.click()

In [ ]:
def get_items():
    try:
        list_of_items = driver.find_element(By.CSS_SELECTOR, 'ul.ba608fb8.de8df3a3')
        items = list_of_items.find_elements(By.TAG_NAME, 'li')
        print(len(items))
        data = []
        for i in items:
            try:
                price = i.find_element(By.CSS_SELECTOR, 'div[aria-label="Price"]').text
                name = i.find_element(By.CSS_SELECTOR, 'div[aria-label="Title"]').text
                location = i.find_element(By.CSS_SELECTOR, 'span[aria-label="Location"]').text
                creation_date = i.find_element(By.CSS_SELECTOR, 'span[aria-label="Creation date"]').text
                data.append([name, price, location, creation_date])
            except NoSuchElementException:
                print("Some elements not found in this item. Skipping.")
                continue
        return data
    except NoSuchElementException:
        print("List of items not found.")
        return []

In [ ]:
driver = webdriver.Chrome()
driver.implicitly_wait(20)
land_page(BASE_URL)
change_location('location')
search_item('item')

In [ ]:
data = get_items()

In [ ]:
df = pd.DataFrame(data, columns=['name', 'price', 'location', 'creation_date'])

In [ ]:
df.min()    

In [ ]:
df['price'] = df['price'].str.replace('Rs', '').str.replace(',', '')

# Convert the 'price' column to numeric
df['price'] = pd.to_numeric(df['price'])

In [ ]:
from datetime import datetime, timedelta

def convert_relative_time(relative_time):
    if 'hour' in relative_time:
        hours = int(relative_time.split()[0])
        return datetime.now() - timedelta(hours=hours)
    elif 'day' in relative_time:
        days = int(relative_time.split()[0])
        return datetime.now() - timedelta(days=days)
    else:
        return None  # Handle other cases as needed


In [ ]:
df['creation_date'] = df['creation_date'].apply(convert_relative_time)

In [ ]:
df

In [ ]:
df.to_csv('item', index=False)